# **CoffeeRoasting_using_numpy**

This project focuses on building a prediction model, "Coffee Roasting predictive model ". The model uses two features, Temperature in Celsius and Duration in minutes, to predict the probability of achieving a good coffee roast. The project follows the guidelines provided by "Coffee Roasting at Home," aiming for a duration between 12 and 15 minutes and a temperature between 175 and 260 degrees Celsius. The model outputs probabilities, and a threshold of 0.5 is applied to make binary decisions.

In [72]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential


### Loading DATA

In [73]:
def load_coffee_data():
    """ Creates a coffee roasting data set.
        roasting duration: 12-15 minutes is best
        temperature range: 175-260C is best
    """
    rng = np.random.default_rng(2)
    X = rng.random(400).reshape(-1,2)
    X[:,1] = X[:,1] * 4 + 11.5          # 12-15 min is best
    X[:,0] = X[:,0] * (285-150) + 150  # 350-500 F (175-260 C) is best
    Y = np.zeros(len(X))

    i=0
    for t,d in X:
        y = -3/(260-175)*t + 21
        if (t > 175 and t < 260 and d > 12 and d < 15 and d<=y ):
            Y[i] = 1
        else:
            Y[i] = 0
        i += 1

    return (X, Y.reshape(-1,1))

In [74]:
X,Y = load_coffee_data();
print(X.shape, Y.shape)

(200, 2) (200, 1)


### Normalizing data

In [75]:
print(f"Temperature Max, Min pre normalization: {np.max(X[:,0]):0.2f}, {np.min(X[:,0]):0.2f}")
print(f"Duration    Max, Min pre normalization: {np.max(X[:,1]):0.2f}, {np.min(X[:,1]):0.2f}")

Temperature Max, Min pre normalization: 284.99, 151.32
Duration    Max, Min pre normalization: 15.45, 11.51


In [76]:
# normalize data
norm_1 = tf.keras.layers.Normalization(axis=1)
norm_1.adapt(X)
Xn = norm_1(X)
print(f"Temperature Max, Min post normalization: {np.max(Xn[:,0]):0.2f}, {np.min(Xn[:,0]):0.2f}")
print(f"Duration    Max, Min post normalization: {np.max(Xn[:,1]):0.2f}, {np.min(Xn[:,1]):0.2f}")

Temperature Max, Min post normalization: 1.66, -1.69
Duration    Max, Min post normalization: 1.79, -1.70


In [77]:
def sigmoid(x):
  return 1 / (1 + np.exp(-x))

### Defining our own Dense and Sequential Function

In [78]:
def Dense(a_in,W,b):
  units=W.shape[1]
  a_out=np.zeros(units)
  for j in range(units):
    w=W[:,j]
    z=np.dot(a_in,w)+b[j]
    a_out[j]=sigmoid(z)
  return(a_out)


In [79]:
def Sequential(x,w1,b1,w2,b2):
  a1=Dense(x,w1,b1)
  a2=Dense(a1,w2,b2)
  return(a2)

### Given WEIGTHS and BIAS's

In [80]:
W1 = np.array( [[-8.93,  0.29, 12.9 ], [-0.1,  -7.32, 10.81]] )
b1 = np.array( [-9.82, -9.28,  0.96] )
W2 = np.array( [[-31.18], [-27.59], [-32.56]] )
b2 = np.array( [15.41] )

### Prediction function

In [81]:
def prediction(X,W1,b1,W2,b2):
  m=X.shape[0]
  p=np.zeros((m,1))
  for i in range(m):
    p[i,0]=Sequential(X[i],W1,b1,W2,b2)

  return(p)

# TESTING THE TRAINED MODEL

In [82]:
X_tst = np.array([
    [200,13.9],  # postive example
    [200,17]])   # negative example

In [83]:

X_n=norm_1(X_tst) #normalizing
q = prediction(X_n,W1,b1,W2,b2)
print(q)

[[9.71931351e-01]
 [3.28978710e-08]]


/tmp/ipython-input-170590882.py:5: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  p[i,0]=Sequential(X[i],W1,b1,W2,b2)


#### A Decision function to convert the probabilities to a DECISION

In [84]:
def decision(prediction):
  y=np.zeros_like(prediction)
  for i in range(len(prediction)):
    if prediction[i]>=0.5:
      y[i]=1
    else:
      y[i]=0
  print("DECISIONS = ",y)

In [85]:
decision(q)


DECISIONS =  [[1.]
 [0.]]
